# 🎯 Spark SQL Basics: SQL Interface for DataFrames

**Time to complete:** 35 minutes  
**Difficulty:** Intermediate  
**Prerequisites:** DataFrame basics, SQL knowledge

---

## 🎯 Learning Objectives

By the end of this notebook, you will master:
- ✅ **Spark SQL fundamentals** - SQL queries on DataFrames
- ✅ **Creating and using views** - Temporary and global views
- ✅ **SQL functions** - Built-in SQL functions and operations
- ✅ **Complex queries** - Joins, subqueries, CTEs
- ✅ **Integration patterns** - SQL + DataFrame API mixing
- ✅ **Performance considerations** - When to use SQL vs DataFrames

**Spark SQL brings familiar SQL syntax to distributed data processing!**

---

## 🔍 Understanding Spark SQL

**Spark SQL** provides a SQL interface to work with structured data in Spark. It allows you to:

- **Write SQL queries** on DataFrames and RDDs
- **Create databases and tables** in Spark
- **Use familiar SQL syntax** for data manipulation
- **Mix SQL and DataFrame API** in the same application
- **Access data sources** through SQL

### Architecture:
```
SQL Query → Catalyst Optimizer → Logical Plan → Physical Plan → Execution
```

**Spark SQL uses the same execution engine as DataFrame API - they're interchangeable!**

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, round as round_func
import pyspark.sql.functions as F

spark = SparkSession.builder \
    .appName("Spark_SQL_Basics") \
    .master("local[*]") \
    .getOrCreate()

print(f"✅ Spark ready - Version: {spark.version}")

# Enable Spark SQL
spark.sql("SET spark.sql.adaptive.enabled = true")
print("Spark SQL optimizations enabled")

# Create sample datasets
employees_data = [
    (1, "Alice", "Engineering", 75000, "2023-01-15"),
    (2, "Bob", "Sales", 65000, "2023-02-20"),
    (3, "Charlie", "Engineering", 85000, "2023-03-10"),
    (4, "Diana", "HR", 55000, "2023-04-05"),
    (5, "Eve", "Sales", 70000, "2023-05-12"),
    (6, "Frank", "Finance", 80000, "2023-06-18"),
    (7, "Grace", "Engineering", 78000, "2023-07-22"),
    (8, "Henry", "Sales", 72000, "2023-08-30")
]

departments_data = [
    ("Engineering", "Building software", "New York", 10),
    ("Sales", "Selling products", "Chicago", 8),
    ("HR", "Managing people", "Boston", 3),
    ("Finance", "Financial management", "New York", 5),
    ("Marketing", "Brand promotion", "Los Angeles", 6)
]

projects_data = [
    (101, "Website Redesign", "Engineering", 3, "2023-12-01"),
    (102, "Sales CRM", "Sales", 2, "2023-11-15"),
    (103, "Talent Acquisition", "HR", 1, "2023-10-20"),
    (104, "Budget Planning", "Finance", 2, "2023-09-10"),
    (105, "Brand Campaign", "Marketing", 4, "2023-08-25")
]

# Create DataFrames
employees_df = spark.createDataFrame(employees_data, 
    ["emp_id", "name", "department", "salary", "hire_date"])

departments_df = spark.createDataFrame(departments_data, 
    ["dept_name", "description", "location", "team_size"])

projects_df = spark.createDataFrame(projects_data, 
    ["project_id", "project_name", "department", "priority", "deadline"])

print("📊 Sample DataFrames:")
print("\nEmployees:")
employees_df.show()

print("Departments:")
departments_df.show()

print("Projects:")
projects_df.show()

## 🎯 Creating Views and Running SQL Queries

### Creating Temporary Views

In [ ]:
# Create temporary views for SQL queries
print("🎯 CREATING VIEWS AND SQL QUERIES")
print("=" * 50)

# Create temporary views (available in current session)
employees_df.createOrReplaceTempView("employees")
departments_df.createOrReplaceTempView("departments")
projects_df.createOrReplaceTempView("projects")

print("✅ Temporary views created")

# List available tables
tables_df = spark.sql("SHOW TABLES")
print("\nAvailable tables:")
tables_df.show()

# Basic SQL SELECT query
basic_query = spark.sql("""
    SELECT emp_id, name, department, salary
    FROM employees
    WHERE salary > 70000
    ORDER BY salary DESC
""")

print("\nBasic SQL query (salary > 70k):")
basic_query.show()

# SQL with aggregations
agg_query = spark.sql("""
    SELECT 
        department,
        COUNT(*) as employee_count,
        AVG(salary) as avg_salary,
        MAX(salary) as max_salary,
        MIN(salary) as min_salary
    FROM employees
    GROUP BY department
    ORDER BY avg_salary DESC
""")

print("\nSQL aggregation by department:")
agg_query.show()

### SQL Functions and Expressions

In [ ]:
# SQL functions and expressions
print("🧮 SQL FUNCTIONS AND EXPRESSIONS")
print("=" * 50)

# String functions
string_functions = spark.sql("""
    SELECT 
        name,
        UPPER(name) as upper_name,
        LOWER(name) as lower_name,
        LENGTH(name) as name_length,
        SUBSTRING(name, 1, 3) as first_3_chars,
        CONCAT(name, ' (', department, ')') as full_name
    FROM employees
""")

print("String functions:")
string_functions.show()

# Mathematical and conditional functions
math_functions = spark.sql("""
    SELECT 
        name,
        salary,
        ROUND(salary * 0.10, 2) as bonus,
        salary * 1.10 as salary_with_bonus,
        CASE 
            WHEN salary >= 80000 THEN 'High'
            WHEN salary >= 60000 THEN 'Medium'
            ELSE 'Entry'
        END as salary_tier,
        YEAR(hire_date) as hire_year,
        MONTH(hire_date) as hire_month
    FROM employees
""")

print("\nMathematical and conditional functions:")
math_functions.show()

# Window functions in SQL
window_functions = spark.sql("""
    SELECT 
        name,
        department,
        salary,
        ROW_NUMBER() OVER (PARTITION BY department ORDER BY salary DESC) as dept_rank,
        RANK() OVER (ORDER BY salary DESC) as overall_rank,
        SUM(salary) OVER (PARTITION BY department) as dept_total_salary,
        AVG(salary) OVER (PARTITION BY department) as dept_avg_salary
    FROM employees
""")

print("\nWindow functions in SQL:")
window_functions.show()

## 🔗 SQL Joins

### Inner, Left, and Other Join Types

In [ ]:
# SQL joins
print("🔗 SQL JOINS")
print("=" * 50)

# Inner join
inner_join = spark.sql("""
    SELECT 
        e.name,
        e.department,
        e.salary,
        d.description,
        d.location,
        d.team_size
    FROM employees e
    INNER JOIN departments d ON e.department = d.dept_name
    ORDER BY e.salary DESC
""")

print("Inner join (employees + departments):")
inner_join.show()

# Left join
left_join = spark.sql("""
    SELECT 
        e.name,
        e.department as emp_dept,
        e.salary,
        d.dept_name,
        d.location,
        d.team_size
    FROM employees e
    LEFT JOIN departments d ON e.department = d.dept_name
    ORDER BY e.name
""")

print("\nLeft join (all employees, matching departments):")
left_join.show()

# Find employees without departments
orphaned_employees = spark.sql("""
    SELECT e.name, e.department, e.salary
    FROM employees e
    LEFT JOIN departments d ON e.department = d.dept_name
    WHERE d.dept_name IS NULL
""")

print("\nEmployees without departments:")
orphaned_employees.show()

# Right join
right_join = spark.sql("""
    SELECT 
        e.name,
        d.dept_name,
        d.description,
        d.location
    FROM employees e
    RIGHT JOIN departments d ON e.department = d.dept_name
    ORDER BY d.dept_name
""")

print("\nRight join (all departments, matching employees):")
right_join.show()

### Complex Multi-Table Joins

In [ ]:
# Complex multi-table joins
print("🔀 COMPLEX MULTI-TABLE JOINS")
print("=" * 50)

# Three-way join
three_way_join = spark.sql("""
    SELECT 
        e.name as employee_name,
        e.salary,
        d.dept_name,
        d.location,
        p.project_name,
        p.priority,
        p.deadline
    FROM employees e
    INNER JOIN departments d ON e.department = d.dept_name
    LEFT JOIN projects p ON e.department = p.department
    ORDER BY e.salary DESC, p.priority ASC
""")

print("Three-way join (employees → departments → projects):")
three_way_join.show()

# Subqueries
subquery_example = spark.sql("""
    SELECT 
        dept_name,
        location,
        team_size,
        avg_salary
    FROM (
        SELECT 
            department,
            AVG(salary) as avg_salary,
            COUNT(*) as emp_count
        FROM employees
        GROUP BY department
    ) dept_stats
    INNER JOIN departments d ON dept_stats.department = d.dept_name
    WHERE dept_stats.avg_salary > 65000
    ORDER BY dept_stats.avg_salary DESC
""")

print("\nSubquery example (departments with high avg salary):")
subquery_example.show()

# CTE (Common Table Expression)
cte_example = spark.sql("""
    WITH dept_stats AS (
        SELECT 
            department,
            COUNT(*) as employee_count,
            AVG(salary) as avg_salary,
            MAX(salary) as max_salary
        FROM employees
        GROUP BY department
    ),
    high_performers AS (
        SELECT 
            name,
            department,
            salary
        FROM employees
        WHERE salary > 70000
    )
    SELECT 
        ds.department,
        ds.employee_count,
        ROUND(ds.avg_salary, 2) as avg_salary,
        hp.name as top_employee,
        hp.salary as top_salary
    FROM dept_stats ds
    LEFT JOIN high_performers hp ON ds.department = hp.department
    ORDER BY ds.avg_salary DESC
""")

print("\nCTE example (department stats with top performers):")
cte_example.show()

## 🎨 Mixing SQL and DataFrame API

### SQL + DataFrame Integration

In [ ]:
# Mixing SQL and DataFrame API
print("🎨 MIXING SQL AND DATAFRAME API")
print("=" * 50)

# Create DataFrame using SQL, then use DataFrame API
sql_result = spark.sql("""
    SELECT 
        department,
        name,
        salary,
        ROW_NUMBER() OVER (PARTITION BY department ORDER BY salary DESC) as dept_rank
    FROM employees
""")

# Continue with DataFrame API
mixed_result = sql_result \
    .filter(col("dept_rank") <= 2) \
    .withColumn("salary_category", 
                when(col("salary") >= 75000, "High")
                .otherwise("Regular")) \
    .groupBy("department", "salary_category") \
    .agg(F.count("name").alias("count"), F.avg("salary").alias("avg_salary")) \
    .orderBy("department", "salary_category")

print("SQL query + DataFrame API processing:")
mixed_result.show()

# Create DataFrame with API, register as view, then query with SQL
api_df = employees_df \
    .withColumn("experience_level", 
                when(col("salary") >= 80000, "Senior")
                .when(col("salary") >= 60000, "Mid")
                .otherwise("Junior")) \
    .withColumn("annual_bonus", col("salary") * 0.15)

# Register as temporary view
api_df.createOrReplaceTempView("enriched_employees")

# Query with SQL
sql_on_api_df = spark.sql("""
    SELECT 
        experience_level,
        COUNT(*) as count,
        ROUND(AVG(salary), 2) as avg_salary,
        ROUND(AVG(annual_bonus), 2) as avg_bonus,
        ROUND(SUM(salary + annual_bonus), 2) as total_compensation
    FROM enriched_employees
    GROUP BY experience_level
    ORDER BY avg_salary DESC
""")

print("\nDataFrame API processing + SQL aggregation:")
sql_on_api_df.show()

# Dynamic SQL generation
def generate_dynamic_sql(dept_filter=None):
    base_query = """
        SELECT department, COUNT(*) as count, AVG(salary) as avg_salary
        FROM employees
    """
    
    if dept_filter:
        query = f"{base_query} WHERE department = '{dept_filter}' GROUP BY department"
    else:
        query = f"{base_query} GROUP BY department"
    
    return spark.sql(query)

print("\nDynamic SQL generation:")
generate_dynamic_sql().show()
print("\nFiltered for Engineering:")
generate_dynamic_sql("Engineering").show()

## 📊 Advanced SQL Features

### Complex Queries and Analytics

In [ ]:
# Advanced SQL features
print("📊 ADVANCED SQL FEATURES")
print("=" * 50)

# Complex analytics query
analytics_query = spark.sql("""
    WITH employee_stats AS (
        SELECT 
            department,
            COUNT(*) as emp_count,
            AVG(salary) as avg_salary,
            STDDEV(salary) as salary_stddev,
            MIN(salary) as min_salary,
            MAX(salary) as max_salary
        FROM employees
        GROUP BY department
    ),
    department_projects AS (
        SELECT 
            department,
            COUNT(*) as project_count,
            AVG(priority) as avg_priority,
            MAX(deadline) as latest_deadline
        FROM projects
        GROUP BY department
    )
    SELECT 
        COALESCE(es.department, dp.department) as department,
        es.emp_count,
        ROUND(es.avg_salary, 2) as avg_salary,
        ROUND(es.salary_stddev, 2) as salary_variation,
        dp.project_count,
        ROUND(dp.avg_priority, 1) as avg_project_priority,
        dp.latest_deadline,
        CASE 
            WHEN es.avg_salary > 75000 THEN 'High Cost'
            WHEN es.avg_salary > 60000 THEN 'Medium Cost'
            ELSE 'Low Cost'
        END as cost_category,
        CASE 
            WHEN dp.project_count > 0 THEN 'Active'
            ELSE 'Inactive'
        END as project_status
    FROM employee_stats es
    FULL OUTER JOIN department_projects dp ON es.department = dp.department
    ORDER BY es.avg_salary DESC NULLS LAST
""")

print("Advanced analytics query (employees + projects):")
analytics_query.show(truncate=False)

# Pivot-like operations
pivot_query = spark.sql("""
    SELECT 
        department,
        COUNT(CASE WHEN salary >= 80000 THEN 1 END) as high_earners,
        COUNT(CASE WHEN salary BETWEEN 60000 AND 79999 THEN 1 END) as mid_earners,
        COUNT(CASE WHEN salary < 60000 THEN 1 END) as entry_earners,
        ROUND(AVG(CASE WHEN salary >= 80000 THEN salary END), 2) as avg_high_salary,
        ROUND(AVG(CASE WHEN salary < 60000 THEN salary END), 2) as avg_entry_salary
    FROM employees
    GROUP BY department
    ORDER BY department
""")

print("\nPivot-style analysis by salary ranges:")
pivot_query.show()

# Time-based analytics
time_analysis = spark.sql("""
    SELECT 
        YEAR(hire_date) as hire_year,
        MONTH(hire_date) as hire_month,
        COUNT(*) as hires,
        ROUND(AVG(salary), 2) as avg_salary,
        COUNT(DISTINCT department) as departments_hiring,
        SUM(COUNT(*)) OVER (ORDER BY YEAR(hire_date), MONTH(hire_date)) as cumulative_hires
    FROM employees
    GROUP BY YEAR(hire_date), MONTH(hire_date)
    ORDER BY hire_year, hire_month
""")

print("\nTime-based hiring analytics:")
time_analysis.show()

## ⚡ Performance Considerations

### SQL vs DataFrame API Performance

In [ ]:
# Performance considerations
print("⚡ SQL VS DATAFRAME API PERFORMANCE")
print("=" * 50)

# Create larger dataset for performance testing
large_data = [
    (i, f"Employee_{i}", f"Dept_{(i%10)+1}", 50000 + (i % 50000), f"2023-{(i%12)+1:02d}-01")
    for i in range(50000)
]

large_df = spark.createDataFrame(large_data, 
    ["emp_id", "name", "department", "salary", "hire_date"])
large_df.createOrReplaceTempView("large_employees")

print(f"Large dataset: {large_df.count():,} rows")

# Test 1: Simple filtering
print("\n=== Test 1: Simple Filtering ===")

# SQL approach
import time
start_time = time.time()
sql_result = spark.sql("""
    SELECT * FROM large_employees 
    WHERE salary > 75000 AND department = 'Dept_1'
""")
sql_count = sql_result.count()
sql_time = time.time() - start_time

# DataFrame API approach
start_time = time.time()
df_result = large_df.filter((col("salary") > 75000) & (col("department") == "Dept_1"))
df_count = df_result.count()
df_time = time.time() - start_time

print(f"SQL approach: {sql_time:.3f} seconds ({sql_count} rows)")
print(f"DataFrame API: {df_time:.3f} seconds ({df_count} rows)")
print(f"Results match: {sql_count == df_count}")

# Test 2: Complex aggregation
print("\n=== Test 2: Complex Aggregation ===")

# SQL approach
start_time = time.time()
sql_agg = spark.sql("""
    SELECT 
        department,
        COUNT(*) as emp_count,
        AVG(salary) as avg_salary,
        MAX(salary) as max_salary,
        COUNT(CASE WHEN salary > 75000 THEN 1 END) as high_earners
    FROM large_employees
    GROUP BY department
    HAVING COUNT(*) > 4000
    ORDER BY avg_salary DESC
""")
sql_agg_count = sql_agg.count()
sql_agg_time = time.time() - start_time

# DataFrame API approach
start_time = time.time()
df_agg = large_df.groupBy("department").agg(
    F.count("*").alias("emp_count"),
    F.avg("salary").alias("avg_salary"),
    F.max("salary").alias("max_salary"),
    F.sum(when(col("salary") > 75000, 1).otherwise(0)).alias("high_earners")
).filter(col("emp_count") > 4000).orderBy(col("avg_salary").desc())
df_agg_count = df_agg.count()
df_agg_time = time.time() - start_time

print(f"SQL aggregation: {sql_agg_time:.3f} seconds ({sql_agg_count} groups)")
print(f"DataFrame agg: {df_agg_time:.3f} seconds ({df_agg_count} groups)")

# Performance insights
print("\n📊 PERFORMANCE INSIGHTS:")
print("- SQL and DataFrame API use the same execution engine")
print("- Performance differences are usually negligible")
print("- Choose based on team preference and use case")
print("- SQL is often more readable for complex joins")
print("- DataFrame API is better for programmatic operations")
print("- Catalyst optimizer handles both equally well")

## 🚨 Common Mistakes and Best Practices

In [ ]:
# Common mistakes and best practices
print("🚨 COMMON MISTAKES AND BEST PRACTICES")
print("=" * 60)

# Mistake 1: Not creating views before querying
print("❌ Mistake: Querying DataFrames without views")
try:
    # This won't work - employees_df is not a table
    bad_query = spark.sql("SELECT * FROM employees_df")
    bad_query.show()
except Exception as e:
    print(f"Error: {str(e)[:80]}...")

print("\n✅ Solution: Create views first")
employees_df.createOrReplaceTempView("employees_table")
good_query = spark.sql("SELECT * FROM employees_table LIMIT 3")
good_query.show()

# Mistake 2: SQL injection vulnerability
print("\n❌ Mistake: SQL injection in dynamic queries")
def unsafe_query(user_input):
    # DON'T DO THIS - vulnerable to SQL injection
    return spark.sql(f"SELECT * FROM employees WHERE name = '{user_input}'")

print("\n✅ Solution: Use parameterized queries or DataFrame API")
def safe_query(name_filter):
    return employees_df.filter(col("name") == name_filter)

safe_query("Alice").show()

# Mistake 3: Forgetting NULL handling in SQL
print("\n❌ Mistake: NULL handling in SQL")
null_query = spark.sql("""
    SELECT department, AVG(salary) as avg_salary
    FROM employees
    GROUP BY department
""")
print("Query works but NULLs are ignored in AVG:")
null_query.show()

# Mistake 4: Performance - not using EXPLAIN
print("\n❌ Mistake: Not checking query plans")
print("Always use EXPLAIN to understand execution plans:")

explain_query = spark.sql("""
    SELECT e.name, d.location
    FROM employees e
    INNER JOIN departments d ON e.department = d.dept_name
    WHERE e.salary > 60000
""")

print("\nQuery execution plan:")
explain_query.explain(mode="formatted")

# Best practices summary
print("\n🎯 SQL BEST PRACTICES:")
print("1. Create views before complex queries")
print("2. Use CTEs for complex multi-step logic")
print("3. Always check execution plans with EXPLAIN")
print("4. Handle NULLs explicitly in aggregations")
print("5. Use appropriate join types for your use case")
print("6. Mix SQL and DataFrame API when beneficial")
print("7. Validate query results against expectations")
print("8. Use descriptive aliases for complex queries")

## 🎯 Key Takeaways

### What You Learned:
- ✅ **`spark.sql()`** - Execute SQL queries on DataFrames
- ✅ **`createOrReplaceTempView()`** - Register DataFrames as SQL tables
- ✅ **SQL functions** - String, math, date, and aggregate functions
- ✅ **Joins in SQL** - INNER, LEFT, RIGHT, FULL OUTER joins
- ✅ **Complex queries** - CTEs, subqueries, window functions
- ✅ **SQL + DataFrame mixing** - Best of both worlds
- ✅ **Performance optimization** - SQL vs DataFrame API comparison

### SQL Capabilities in Spark:
- 🔸 **Standard SQL** - SELECT, FROM, WHERE, GROUP BY, HAVING, ORDER BY
- 🔸 **Joins** - All major join types (INNER, LEFT, RIGHT, FULL)
- 🔸 **Subqueries** - Nested queries and CTEs
- 🔸 **Window functions** - ROW_NUMBER, RANK, SUM OVER, etc.
- 🔸 **Built-in functions** - 200+ SQL functions available
- 🔸 **User-defined functions** - Custom SQL functions

### When to Use SQL vs DataFrame API:
- 🔸 **Use SQL when**: Team prefers SQL, complex joins, reporting queries
- 🔸 **Use DataFrame API when**: Programmatic logic, complex transformations, UDFs
- 🔸 **Mix both when**: SQL for data access, DataFrame for processing

### Performance Notes:
- 🔸 **Same execution engine** - SQL and DataFrame API performance is identical
- 🔸 **Catalyst optimization** - Both benefit from the same query optimizer
- 🔸 **Choose based on preference** - Use what your team knows best

### Common Patterns:
- 🔸 `spark.sql("SELECT ... FROM table")` - Basic SQL query
- 🔸 `df.createOrReplaceTempView("table")` - Register DataFrame as table
- 🔸 `spark.sql("CREATE TEMP VIEW view_name AS SELECT ...")` - Create views
- 🔸 `WITH cte_name AS (...) SELECT ... FROM cte_name` - CTE pattern
- 🔸 `spark.sql(query).toPandas()` - Convert results to Pandas

---

## 🚀 Next Steps

Now that you master Spark SQL basics, you're ready for:

1. **Temporary vs Global Views** - View management and persistence
2. **SQL vs DataFrame API** - When to use each approach
3. **SQL Optimization** - Advanced query optimization techniques
4. **Integration Patterns** - SQL in production Spark applications

**Spark SQL brings the power of SQL to distributed computing!**

---

**🎉 Congratulations! You now have SQL superpowers in Spark!**